# Project Phase 4 Appendix: Data Cleaning Description
### Samantha Rubin, Hanwen Lin, Liz Espinoza, and Hanna Kang

Below, we import all the needed packages as well as the initial datasets:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

raw_covid = pd.read_csv("jrc-covid-19-world-latest.csv")
gini_data = pd.read_csv("gini_data2.csv")
gdp_data = pd.read_csv("gdp_data.csv")
pop_data = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_2106202.csv")
pop_den = pd.read_csv("pop_density.csv")
unemploy_rates = pd.read_csv("unemployment_rates.csv")

For gdp_data and gini_data, what should be the column labels are actually stored in the third row, and the data starts on the fourth row. This is fixed below:

In [2]:
gdp_data.columns = gdp_data.loc[3]
gdp_data = gdp_data.drop(index = [0,1,2,3])

We then work on removing unnecessary columns and renaming them:

In [3]:
new_colnames = [i.lower() for i in raw_covid.columns]
covid_cases = raw_covid.copy()
for i in covid_cases.columns:
    covid_cases.columns = new_colnames
    
#dropping unnecessary columns:
covid_cases = covid_cases[['date', 'iso3', 'continent', 'countryname'
                           , 'cumulativepositive', 'cumulativedeceased'
                           , 'currentlypositive']]
#renaming columns
covid_cases.rename(columns = {'iso3':'country_code','countryname':'country_name'
                              , 'cumulativepositive':'cumulative_pos', 'cumulativedeceased':'cumulative_dec'
                              , 'currentlypositive':'curr_pos'}, inplace = True)



#clean gdp data, drop unecessary columns and rename them
new_colnames = []
gdp = gdp_data.copy()
gdp.rename(columns = {"Country Name":"country_name","Country Code":"country_code",2019:"gdp_per_cap"}
           , inplace = True)
gdp = gdp[["country_name","country_code","gdp_per_cap"]]

#drop NaNs in gdp
gdp = gdp.dropna(axis = 0,subset=["gdp_per_cap"])

#clean gini column names
gini_data.rename(columns = {"Country Code":"country_code",
                          "Country":"country","Value":"gini_index",'Year':"gini_year"},inplace = True)

#clean population data
pop_data.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2019":"population"}
                ,inplace = True)
pop_data = pop_data[["country_code",'country_name','population']]

#clean population density 
pop_den.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2018":"density"}
               ,inplace = True)
pop_den = pop_den[['country_code','country_name','density']]

#clean unemployment rates 
unemploy_rates.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2019":"rates"}
                      ,inplace = True)
unemploy_rates = unemploy_rates[['country_code','country_name','rates']]

Below, we matched countries via country codes, only keeping the countries that had GDP, Gini Index, population density, and unemployment rate values. We printed the length to see how many countries were dropped:

In [4]:
#adding gdp 
covid_gdp = pd.merge(left = covid_cases, right = gdp, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp))

#adding gini
covid_gdp_gini = pd.merge(left = covid_gdp, right = gini_data, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding population 
covid_gdp_gini = pd.merge(left = covid_gdp_gini, right = pop_data, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding population density
covid_gdp_gini = pd.merge(left = covid_gdp_gini, right = pop_den, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding unemployment rates
covid_data = pd.merge(left = covid_gdp_gini, right = unemploy_rates, left_on = "country_code",right_on = "country_code")
print(len(covid_data))


#removing duplicate columns
covid_data = covid_data.drop(columns = ["country",'country_name_y',"country_name"])
covid_data.rename(columns = {"country_name_x":"country_name"},inplace = True)

166
140
140
136
136


Below, we create some new columns that will allow us to further analyze the dataset:

In [5]:
#creating a new column for the ratio of covid deaths to covid positives
covid_data['dec_pos_ratio'] = covid_data['cumulative_dec'] / covid_data['cumulative_pos']

#creating two columns: cumulative COVID positives per capita, and cumulative COVID deaths per capita
cpos_norm = []
for i in range(len(covid_data.cumulative_pos)):
    cpos_norm.append(covid_data.cumulative_pos[i]/covid_data.population[i])
covid_data['cpos_norm'] = cpos_norm

cdec_norm = []
for i in range(len(covid_data.cumulative_dec)):
    cdec_norm.append(covid_data.cumulative_dec[i]/covid_data.population[i])
covid_data['cdec_norm'] = cdec_norm

Below, we further clean columns by dropping uneccesary data and renaming:

In [6]:
#cleaning columns
covid_data.rename(columns = {'density':'pop_density', 'rates': 'unemploy_rates'}, inplace = True)
covid_data.drop(labels = 'country_name', axis = 1, inplace = True)
covid_data.head()

,date,country_code,continent,cumulative_pos,cumulative_dec,curr_pos,gdp_per_cap,gini_index,gini_year,population,pop_density,unemploy_rates,dec_pos_ratio,cpos_norm,cdec_norm
0,2021-03-13,ALB,EU,116821,2030,34970.0,5353.244856,29.0,2012,2854191.0,104.6122628,11.47000027,0.017377,0.040930,0.000711
1,2021-02-15,DZA,AF,110711,2939,107772.0,3973.964072,27.6,2011,43053054.0,17.73006763,11.81000042,0.026547,0.002572,0.000068
2,2021-02-15,AGO,AF,20366,492,19874.0,2790.726615,42.7,2008,31825295.0,24.71305206,6.929999828,0.024158,0.000640,0.000015
3,2021-02-15,ARG,SA,2029008,50327,1978681.0,9912.281809,41.2,2017,44938712.0,16.2585101,9.840000153,0.024804,0.045151,0.001120
4,2021-02-15,AUS,OC,28900,909,27991.0,55060.326100,35.8,2014,25364307.0,3.24787091,5.159999847,0.031453,0.001139,0.000036


Then we try and clean up the unemployment rates column:

In [7]:
#trying to get rid of strings that are in the unemployment columns 
for i in range(len(covid_data['unemploy_rates'])):
    if "*" in covid_data.unemploy_rates[i]:
        covid_data.loc[i,"unemploy_rates"]=np.nan
    else:
        covid_data.loc[i,"unemploy_rates"]=float(covid_data.unemploy_rates[i])
covid_data.dropna(axis=0, subset=["unemploy_rates"], inplace=True)

Below we display the final cleaned dataset:

In [8]:
covid_data.head()

,date,country_code,continent,cumulative_pos,cumulative_dec,curr_pos,gdp_per_cap,gini_index,gini_year,population,pop_density,unemploy_rates,dec_pos_ratio,cpos_norm,cdec_norm
0,2021-03-13,ALB,EU,116821,2030,34970.0,5353.244856,29.0,2012,2854191.0,104.6122628,11.47,0.017377,0.040930,0.000711
1,2021-02-15,DZA,AF,110711,2939,107772.0,3973.964072,27.6,2011,43053054.0,17.73006763,11.81,0.026547,0.002572,0.000068
2,2021-02-15,AGO,AF,20366,492,19874.0,2790.726615,42.7,2008,31825295.0,24.71305206,6.93,0.024158,0.000640,0.000015
3,2021-02-15,ARG,SA,2029008,50327,1978681.0,9912.281809,41.2,2017,44938712.0,16.2585101,9.84,0.024804,0.045151,0.001120
4,2021-02-15,AUS,OC,28900,909,27991.0,55060.326100,35.8,2014,25364307.0,3.24787091,5.16,0.031453,0.001139,0.000036


In [9]:
print("Number of countries included: {}".format(covid_data.shape[0]))

Number of countries included: 135


Finally, we convert the covid_data dataframe to a csv file to use in the rest of the project:

In [12]:
covid_data.to_csv('covid_data.csv')